In [3]:
import pandas as pd
import numpy as np

# test.csv 파일을 읽어옵니다.
df = pd.read_csv('test.csv')

# 조건에 맞는 데이터만 필터링합니다.
df = df[(df['license_number'] != 'unknown') & (df['license_number_score'].notna())]

# car_id 별로 데이터를 그룹화하고, 프레임 번호에서 빈 공백을 전부 보정합니다.
def interpolate_license_numbers(group):
    frames = group['frame_nmr'].values
    scores = group['license_number_score'].values
    
    # license_plate_bbox를 문자열에서 리스트로 변환 (공백으로 구분)
    license_plate_bboxes = group['license_plate_bbox'].apply(
        lambda x: list(map(float, x.strip('[]').split()))  # 공백으로 나누도록 수정
    ).values
    
    # 프레임을 0에서부터 연속된 정수로 만든 후, 보간(interpolation)을 합니다.
    full_frames = np.arange(frames.min(), frames.max() + 1)
    interpolated_df = pd.DataFrame({'frame_nmr': full_frames})

    # 점수 데이터 보간
    interpolated_df['license_number_score'] = np.interp(full_frames, frames, scores)
    
    # bbox 보간은 하지 않고 그대로 가져옵니다.
    license_plate_bboxes_dict = dict(zip(frames, license_plate_bboxes))
    
    # 프레임 간 공백이 있을 때, 그 사이의 bbox를 보간하여 채웁니다.
    interpolated_bboxes = []
    for frame in full_frames:
        print(f"처리 중인 프레임: {frame}")  # 디버깅: 현재 프레임을 출력
        if frame in license_plate_bboxes_dict:
            print(f"frame {frame}에 bbox 있음: {license_plate_bboxes_dict[frame]}")  # 디버깅: bbox 값 출력
            interpolated_bboxes.append(license_plate_bboxes_dict[frame])
        else:
            # 앞뒤 프레임의 bbox 차이를 계산하여 보간
            prev_frame = full_frames[full_frames < frame].max() if (full_frames < frame).any() else None
            next_frame = full_frames[full_frames > frame].min() if frame < full_frames.max() else None
            print(f"이전 프레임: {prev_frame}, 다음 프레임: {next_frame}")  # 디버깅: 이전 및 다음 프레임 출력
            
            # prev_frame이 없으면 바로 앞의 프레임을 사용
            if prev_frame is None:  # prev_frame이 없으면 그 전 프레임의 bbox를 사용
                prev_frame = frame - 1  # 바로 앞의 프레임을 사용
            prev_bbox = np.array(license_plate_bboxes_dict.get(prev_frame, []))  # 수정: .get() 사용하여 KeyError 방지
            
            if next_frame is not None and next_frame in license_plate_bboxes_dict:
                next_bbox = np.array(license_plate_bboxes_dict[next_frame])
                
                # 각 컴포넌트별 선형 보간 (x1, y1, x2, y2)
                interpolated_bbox = []
                for prev_val, next_val in zip(prev_bbox, next_bbox):
                    ratio = (frame - prev_frame) / (next_frame - prev_frame)
                    interpolated_bbox.append(prev_val + ratio * (next_val - prev_val))
            else:
                # next_frame이 없다면 이전 프레임의 bbox를 그대로 사용
                interpolated_bbox = prev_bbox
                print(f"bbox 보간을 할 수 없으므로 prev_frame의 bbox 사용: {prev_bbox}")  # 디버깅: 사용된 bbox 출력
                
            interpolated_bboxes.append(interpolated_bbox)  # tolist()는 필요 없음
    
    interpolated_df['license_plate_bbox'] = interpolated_bboxes
    
    # 가장 높은 점수의 license_number를 찾습니다.
    best_license_number = group.loc[group['license_number_score'].idxmax()]['license_number']
    print(f"가장 높은 점수의 license_number: {best_license_number}")  # 디버깅: 최고 점수의 license_number 출력

    # 모든 프레임에 대해 동일한 license_number를 할당합니다.
    interpolated_df['license_number'] = best_license_number
    
    return interpolated_df

# 각 car_id 별로 보간(interpolation)을 수행합니다.
interpolated_df_list = []
for car_id, group in df.groupby('car_id'):
    print(f"Processing car_id: {car_id}")  # 디버깅: 현재 car_id 출력
    interpolated_group = interpolate_license_numbers(group)
    interpolated_group['car_id'] = car_id  # car_id 추가
    interpolated_df_list.append(interpolated_group)

# 모든 car_id에 대한 보간된 데이터프레임을 합칩니다.
final_df = pd.concat(interpolated_df_list)

# 결과를 test2.csv 파일로 저장합니다.
final_df.to_csv('test-interpolated.csv', index=False)

print("Interpolation 완료 후 test2.csv 저장 완료.")


Processing car_id: 27.0
처리 중인 프레임: 462
frame 462에 bbox 있음: [668.3450927734375, 348.2243347167969, 834.7011108398438, 415.2502746582031]
처리 중인 프레임: 463
frame 463에 bbox 있음: [662.1337280273438, 348.9031677246094, 834.8660888671875, 415.0970764160156]
처리 중인 프레임: 464
frame 464에 bbox 있음: [657.94921875, 348.33380126953125, 829.7493896484375, 413.2100830078125]
처리 중인 프레임: 465
frame 465에 bbox 있음: [657.9093627929688, 348.2719421386719, 829.7589721679688, 413.28472900390625]
처리 중인 프레임: 466
이전 프레임: 465, 다음 프레임: 467
처리 중인 프레임: 467
frame 467에 bbox 있음: [648.5001220703125, 344.9659423828125, 826.8666381835938, 412.1864318847656]
처리 중인 프레임: 468
frame 468에 bbox 있음: [643.6536865234375, 343.0175476074219, 823.000732421875, 411.48773193359375]
처리 중인 프레임: 469
frame 469에 bbox 있음: [640.4811401367188, 342.03387451171875, 820.3818969726562, 410.9256591796875]
처리 중인 프레임: 470
frame 470에 bbox 있음: [637.4917602539062, 341.89654541015625, 821.1600952148438, 410.7203063964844]
처리 중인 프레임: 471
frame 471에 bbox 있음: [637.6

In [6]:
import pandas as pd
import numpy as np

# test.csv 파일을 읽어옵니다.
df = pd.read_csv('test.csv')

# 조건에 맞는 데이터만 필터링합니다.
df = df[(df['license_number'] != 'unknown') & (df['license_number_score'].notna())]

# car_id 별로 데이터를 그룹화하고, 프레임 번호에서 빈 공백을 전부 보정합니다.
def interpolate_license_numbers(group):
    frames = group['frame_nmr'].values
    scores = group['license_number_score'].values
    
    # license_plate_bbox를 문자열에서 리스트로 변환 (공백으로 구분)
    license_plate_bboxes = group['license_plate_bbox'].apply(
        lambda x: list(map(float, x.strip('[]').split()))  # 공백으로 나누도록 수정
    ).values
    
    # 프레임을 0에서부터 연속된 정수로 만든 후, 보간(interpolation)을 합니다.
    full_frames = np.arange(frames.min(), frames.max() + 1)
    interpolated_df = pd.DataFrame({'frame_nmr': full_frames})

    # 점수 데이터 보간
    interpolated_df['license_number_score'] = np.interp(full_frames, frames, scores)
    
    # bbox 보간은 하지 않고 그대로 가져옵니다.
    license_plate_bboxes_dict = dict(zip(frames, license_plate_bboxes))
    
    # 프레임 간 공백이 있을 때, 그 사이의 bbox를 보간하여 채웁니다.
    interpolated_bboxes = []
    for frame in full_frames:
        print(f"처리 중인 프레임: {frame}")  # 디버깅: 현재 프레임을 출력
        if frame in license_plate_bboxes_dict:
            print(f"frame {frame}에 bbox 있음: {license_plate_bboxes_dict[frame]}")  # 디버깅: bbox 값 출력
            interpolated_bboxes.append(license_plate_bboxes_dict[frame])
        else:
            # 앞뒤 프레임의 bbox 차이를 계산하여 보간
            prev_frame = full_frames[full_frames < frame].max() if (full_frames < frame).any() else None
            next_frame = full_frames[full_frames > frame].min() if frame < full_frames.max() else None
            print(f"이전 프레임: {prev_frame}, 다음 프레임: {next_frame}")  # 디버깅: 이전 및 다음 프레임 출력
            
            # prev_frame이 없으면 그 다음 프레임을 사용하도록 수정
            if prev_frame is None:
                prev_frame = next_frame  # 이전 프레임이 없으면 다음 프레임으로 대체
                print(f"prev_frame이 없으므로 next_frame({next_frame})을 prev_frame로 사용")  # 디버깅

            prev_bbox = np.array(license_plate_bboxes_dict.get(prev_frame, None))  # 수정: .get() 사용하여 KeyError 방지

            # prev_bbox가 없으면 예외 처리 (빈 bbox로 처리할 수 없음)
            if prev_bbox is None:
                print(f"Error: frame {frame}에 대한 bbox를 찾을 수 없습니다.")
                prev_bbox = [0, 0, 0, 0]  # 빈 bbox로 대체 (기본값 설정)
                print(f"prev_bbox를 기본값 [0, 0, 0, 0]으로 대체: {prev_bbox}")

            print(f"prev_bbox: {prev_bbox}")  # 디버깅: prev_bbox 값 출력
            if next_frame is not None and next_frame in license_plate_bboxes_dict:
                next_bbox = np.array(license_plate_bboxes_dict[next_frame])
                print(f"next_bbox: {next_bbox}")  # 디버깅: next_bbox 값 출력
                
                # 각 컴포넌트별 선형 보간 (x1, y1, x2, y2)
                interpolated_bbox = []
                for prev_val, next_val in zip(prev_bbox, next_bbox):
                    ratio = (frame - prev_frame) / (next_frame - prev_frame)
                    interpolated_bbox.append(prev_val + ratio * (next_val - prev_val))
            else:
                # next_frame이 없다면 이전 프레임의 bbox를 그대로 사용
                interpolated_bbox = prev_bbox
                print(f"bbox 보간을 할 수 없으므로 prev_frame의 bbox 사용: {prev_bbox}")  # 디버깅: 사용된 bbox 출력
                
            interpolated_bboxes.append(interpolated_bbox)  # tolist()는 필요 없음
    
    interpolated_df['license_plate_bbox'] = interpolated_bboxes
    
    # 가장 높은 점수의 license_number를 찾습니다.
    best_license_number = group.loc[group['license_number_score'].idxmax()]['license_number']
    print(f"가장 높은 점수의 license_number: {best_license_number}")  # 디버깅: 최고 점수의 license_number 출력

    # 모든 프레임에 대해 동일한 license_number를 할당합니다.
    interpolated_df['license_number'] = best_license_number
    
    return interpolated_df

# 각 car_id 별로 보간(interpolation)을 수행합니다.
interpolated_df_list = []
for car_id, group in df.groupby('car_id'):
    print(f"Processing car_id: {car_id}")  # 디버깅: 현재 car_id 출력
    interpolated_group = interpolate_license_numbers(group)
    interpolated_group['car_id'] = car_id  # car_id 추가
    interpolated_df_list.append(interpolated_group)

# 모든 car_id에 대한 보간된 데이터프레임을 합칩니다.
final_df = pd.concat(interpolated_df_list)

# 결과를 test2.csv 파일로 저장합니다.
final_df.to_csv('test-interpolated.csv', index=False)

print("Interpolation 완료 후 test2.csv 저장 완료.")


Processing car_id: 27.0
처리 중인 프레임: 462
frame 462에 bbox 있음: [668.3450927734375, 348.2243347167969, 834.7011108398438, 415.2502746582031]
처리 중인 프레임: 463
frame 463에 bbox 있음: [662.1337280273438, 348.9031677246094, 834.8660888671875, 415.0970764160156]
처리 중인 프레임: 464
frame 464에 bbox 있음: [657.94921875, 348.33380126953125, 829.7493896484375, 413.2100830078125]
처리 중인 프레임: 465
frame 465에 bbox 있음: [657.9093627929688, 348.2719421386719, 829.7589721679688, 413.28472900390625]
처리 중인 프레임: 466
이전 프레임: 465, 다음 프레임: 467
prev_bbox: [657.90936279 348.27194214 829.75897217 413.284729  ]
next_bbox: [648.50012207 344.96594238 826.86663818 412.18643188]
처리 중인 프레임: 467
frame 467에 bbox 있음: [648.5001220703125, 344.9659423828125, 826.8666381835938, 412.1864318847656]
처리 중인 프레임: 468
frame 468에 bbox 있음: [643.6536865234375, 343.0175476074219, 823.000732421875, 411.48773193359375]
처리 중인 프레임: 469
frame 469에 bbox 있음: [640.4811401367188, 342.03387451171875, 820.3818969726562, 410.9256591796875]
처리 중인 프레임: 470
frame 470

TypeError: iteration over a 0-d array

In [9]:
import numpy as np

def linear_interpolation(prev_bbox, next_bbox, ratio):
    """
    선형 보간을 사용하여 두 바운딩 박스의 중간 값을 계산하는 함수.
    
    :param prev_bbox: 이전 바운딩 박스 [x1, y1, x2, y2]
    :param next_bbox: 다음 바운딩 박스 [x3, y3, x4, y4]
    :param ratio: 보간 비율 (0 <= ratio <= 1). ratio가 0이면 prev_bbox, 1이면 next_bbox에 가까워짐.
    
    :return: 보간된 바운딩 박스 [x1', y1', x2', y2']
    """
    prev_bbox = np.array(prev_bbox)
    next_bbox = np.array(next_bbox)
    
    # 각 컴포넌트별로 선형 보간
    interpolated_bbox = prev_bbox + ratio * (next_bbox - prev_bbox)
    
    return interpolated_bbox.tolist()

# 예시 사용
prev_bbox = [100, 150, 200, 250]  # 이전 바운딩 박스 (x1, y1, x2, y2)
next_bbox = [120, 170, 220, 270]  # 다음 바운딩 박스 (x1, y1, x2, y2)

# 보간 비율 (0.5는 중간 값)
ratio = 0.5

interpolated_bbox = linear_interpolation(prev_bbox, next_bbox, ratio)

print(f"보간된 바운딩 박스: {interpolated_bbox}")


보간된 바운딩 박스: [110.0, 160.0, 210.0, 260.0]


In [12]:
import pandas as pd

# test.csv 읽기
df = pd.read_csv('test.csv')

# bbox 열의 공백을 쉼표로 변경
df['car_bbox'] = df['car_bbox'].apply(lambda x: x.replace(' ', ',') if isinstance(x, str) else x)
df['license_plate_bbox'] = df['license_plate_bbox'].apply(lambda x: x.replace(' ', ',') if isinstance(x, str) else x)

# 변경된 내용을 저장
df.to_csv('test.csv', index=False)

print("공백이 쉼표로 변경된 파일이 'test_updated.csv'로 저장되었습니다.")


공백이 쉼표로 변경된 파일이 'test_updated.csv'로 저장되었습니다.


In [14]:
import pandas as pd
import numpy as np

def linear_interpolation(prev_bbox, next_bbox, ratio):
    """
    선형 보간을 사용하여 두 바운딩 박스의 중간 값을 계산하는 함수.
    
    :param prev_bbox: 이전 바운딩 박스 [x1, y1, x2, y2]
    :param next_bbox: 다음 바운딩 박스 [x3, y3, x4, y4]
    :param ratio: 보간 비율 (0 <= ratio <= 1). ratio가 0이면 prev_bbox, 1이면 next_bbox에 가까워짐.
    
    :return: 보간된 바운딩 박스 [x1', y1', x2', y2']
    """
    prev_bbox = np.array(prev_bbox)
    next_bbox = np.array(next_bbox)
    
    # 각 컴포넌트별로 선형 보간
    interpolated_bbox = prev_bbox + ratio * (next_bbox - prev_bbox)
    
    return interpolated_bbox.tolist()

def find_nearest_frames(df, missing_idx):
    """
    주어진 프레임에서 앞뒤로 가장 가까운 데이터가 있는 프레임을 찾는 함수.
    
    :param df: 프레임별 바운딩 박스 데이터가 포함된 DataFrame.
    :param missing_idx: 공백이 있는 프레임의 인덱스.
    :return: (prev_frame, next_frame) 데이터 튜플
    """
    prev_idx = missing_idx - 1
    next_idx = missing_idx + 1
    
    # 이전 프레임 찾기
    while prev_idx >= 0 and pd.isnull(df.iloc[prev_idx]['bbox']):
        prev_idx -= 1

    # 다음 프레임 찾기
    while next_idx < len(df) and pd.isnull(df.iloc[next_idx]['bbox']):
        next_idx += 1

    # 이전과 다음 프레임이 모두 유효하면 반환
    if prev_idx >= 0 and next_idx < len(df):
        return df.iloc[prev_idx], df.iloc[next_idx]
    
    return None, None

def interpolate_with_fallback(df):
    """
    공백이 존재하면 선형 보간을 수행하고, 이전/다음 프레임을 찾는 방식으로 보간하는 함수.
    
    :param df: 프레임별 바운딩 박스 데이터가 포함된 DataFrame.
    :return: 보간된 DataFrame.
    """
    interpolated_df = []
    
    for i in range(len(df)):
        current_frame = df.iloc[i]
        
        if pd.notnull(current_frame['license_plate_bbox']):
            interpolated_df.append(current_frame)
        else:
            # 공백인 경우
            prev_frame, next_frame = find_nearest_frames(df, i)
            
            if prev_frame is not None and next_frame is not None:
                prev_bbox = [float(coord) for coord in prev_frame['license_plate_bbox'].split()]
                next_bbox = [float(coord) for coord in next_frame['license_plate_bbox'].split()]
                
                # 프레임 비율 계산 (현재 프레임은 prev_frame과 next_frame 사이의 비율로 보간)
                ratio = (df.iloc[i]['frame'] - prev_frame['frame']) / (next_frame['frame'] - prev_frame['frame'])
                
                # 선형 보간 적용
                interpolated_bbox = linear_interpolation(prev_bbox, next_bbox, ratio)
                
                interpolated_frame = current_frame.copy()
                interpolated_frame['license_plate_bbox'] = ' '.join(map(str, interpolated_bbox))
                
                interpolated_df.append(interpolated_frame)
            else:
                # 보간할 프레임이 없다면 그냥 이전 값 혹은 기본값을 사용
                interpolated_df.append(current_frame)
    
    return pd.DataFrame(interpolated_df)

# test.csv 읽기
df = pd.read_csv('test.csv')

# 보간 수행
interpolated_df = interpolate_with_fallback(df)

# 보간 결과 확인
print(interpolated_df)


      frame_nmr  car_id                                           car_bbox  \
0            35     1.0  [992.7207367496932,287.6132315221208,1183.2023...   
1            36     1.0  [993.8111691922744,287.34976026549236,1183.824...   
2            37     1.0  [994.3410897175274,286.910509994849,1184.86459...   
3            38     1.0  [994.0024062193995,286.57328886217744,1184.582...   
4            39     1.0  [993.9555603670332,286.34476159251597,1184.618...   
...         ...     ...                                                ...   
7597      26536  1035.0  [53.05590960120037,136.28016010411216,1186.713...   
7598      26540  1035.0  [66.35668607545529,98.04636390104815,1309.0611...   
7599      26543  1037.0  [61.076915877627016,111.66588419240122,1388.32...   
7600      26568  1042.0  [543.3720104844144,41.95709695758876,1922.4028...   
7601      26569  1042.0  [470.7683601318711,31.61851219856902,1898.7193...   

                                     license_plate_bbox  \
0   

In [16]:
def replace_license_number_with_max_confidence(df):
    """
    각 car_id 그룹에서 confidence가 가장 높은 번호판 넘버로 replace하는 함수.
    
    :param df: DataFrame containing car_id, license_number, confidence columns.
    :return: 전처리된 DataFrame.
    """
    processed_groups = []

    for car_id, group in df.groupby('car_id'):
        # "Unknown"이나 confidence가 None인 행 제외
        valid_group = group[(group['license_number'] != "Unknown") & (group['license_number_score'].notnull())]
        
        if not valid_group.empty:
            # confidence가 가장 높은 번호판 넘버 선택
            best_row = valid_group.loc[valid_group['license_number_score'].idxmax()]
            best_license_plate_number = best_row['license_number']
            
            # 해당 번호판 넘버로 전체 그룹 replace
            group['license_number'] = best_license_plate_number

        processed_groups.append(group)

    # 모든 그룹 합치기
    return pd.concat(processed_groups)

# CSV 파일 읽기
df = pd.read_csv('test.csv')

# 전처리 수행
processed_df = replace_license_number_with_max_confidence(df)

# 결과를 새로운 CSV로 저장
processed_df.to_csv('processed_test.csv', index=False)

print("번호판 넘버가 전처리된 데이터가 'processed_test.csv'에 저장되었습니다.")


번호판 넘버가 전처리된 데이터가 'processed_test.csv'에 저장되었습니다.
